In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.6 MB/s eta 0:00:00


In [ ]:
import os
import json
import shutil
import tempfile
import time

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib

from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
    ConvertToMultiChannelBasedOnBratsClassesd
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.utils.enums import MetricReduction
from monai.networks.nets import SwinUNETR
from monai import data
from monai.data import decollate_batch
from functools import partial

import torch


print_config()

MONAI version: 1.4.dev2431
Numpy version: 1.26.4
Pytorch version: 2.3.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 951a77d7a7737a3108afa94623a50b87d21eb4a7
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.0.1
scikit-image version: 0.23.2
scipy version: 1.13.1
Pillow version: 9.4.0
Tensorboard version: 2.17.0
gdown version: 4.7.3
TorchVision version: 0.18.1+cu121
tqdm version: 4.66.5
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.1.4
einops version: 0.8.0
transformers version: 4.42.4
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io

In [ ]:
import argparse
import os
from functools import partial
import json

import nibabel as nib
import numpy as np
import torch
from monai import transforms
from monai.transforms import (
    AsDiscrete,
    Activations,
    MapTransform,
)

from monai import data
from monai.inferers import sliding_window_inference
from monai.networks.nets import SwinUNETR

In [ ]:
def datafold_read(datalist, basedir, fold=0, key="validation"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    # tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        # else:
        #     tr.append(d)

    return val

In [ ]:
def get_loader(data_dir, json_list, fold):

    val_transform = transforms.Compose(
        [
            transforms.LoadImaged(keys=["image"],image_only=False),
            #transforms.ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
            transforms.NormalizeIntensityd(
                keys="image", nonzero=True, channel_wise=True),
            transforms.ToTensord(keys=["image"])
        ]
    )

    validation_files = datafold_read(
        datalist=json_list, basedir=data_dir, fold=fold)
    val_ds = data.Dataset(data=validation_files, transform=val_transform)

    val_loader = data.DataLoader(
        val_ds,
        batch_size=1,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
    )

    return val_loader

In [ ]:
def main():
    test_mode = True
    data_dir =  "/content/drive/MyDrive/BRATS/BRATS2024/"
    json_list = "/content/drive/MyDrive/BRATS/validation.json"
    output_directory = "/content/drive/MyDrive/BRATS/output5/"
    test_loader = get_loader(data_dir, json_list, fold=0)
    pretrained_pth = "/content/drive/MyDrive/BRATS/model.pt"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = SwinUNETR(
        img_size=(128, 128, 128),
        in_channels=4,
        out_channels=3,
        feature_size=48,
        use_checkpoint=True,
    )
    # Load the state dictionary from the pretrained model file
    state_dict = torch.load(pretrained_pth)

    # Extract the actual model state dictionary if wrapped with other info
    if 'state_dict' in state_dict:
        state_dict = state_dict['state_dict']

    new_state_dict = {}
    for k, v in state_dict.items():
        # Remove `module.` if present
        name = k[7:] if k.startswith("module.") else k
        new_state_dict[name] = v

    # Load the processed state dictionary into the model
    model.load_state_dict(new_state_dict, strict=False)
    print(new_state_dict)
    print(model)

    model.eval()
    model.to(device)
    model_inferer_test = partial(
        sliding_window_inference,
        roi_size=[128, 128, 128],
        sw_batch_size=1,
        predictor=model,
        overlap=0.5,
    )

    with torch.no_grad():
      for i, batch in enumerate(test_loader):
          image = batch["image"].cuda()
          print(image)
          affine = batch["image_meta_dict"]["original_affine"][0].numpy()
          num = '-'.join(batch["image_meta_dict"]["filename_or_obj"][0].split("/")[-1].split("/")[-1].split("-")[2:4])
          img_name = "{}.nii.gz".format(num)
          print("Inference on case {}".format(img_name))

          prob = torch.sigmoid(model_inferer_test(image))
          seg = prob[0].detach().cpu().numpy()
          seg = (seg > 0.5).astype(np.int8)

          seg_out = np.zeros((seg.shape[1], seg.shape[2], seg.shape[3]))
          seg_out[seg[1] == 1] = 2
          seg_out[seg[0] == 1] = 1
          seg_out[seg[2] == 1] = 3
          nib.save(nib.Nifti1Image(seg_out.astype(np.uint8), affine), os.path.join(output_directory, img_name))
      print("Finished inference!")
      print(output_directory)


In [ ]:
%pip install -q "monai-weekly[einops]"
from einops import rearrange

In [ ]:
main()

Streaming output truncated to the last 5000 lines.
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],


         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0.,